In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_results_list = spark.read.format("delta").load(f"{presentation_container_folder_path}/race_results")\
    .filter(f"file_date = '{v_file_date}'")\
        .select("race_year")\
            .distinct()\
                .collect()

In [0]:
race_year_list= []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
from pyspark.sql.functions import *
race_results_df = spark.read.format("delta").load(f"{presentation_container_folder_path}/race_results")\
    .filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import *
constructor_standings_df = race_results_df\
    .groupBy("race_year","team")\
        .agg(sum("points").alias("total_points"), count(when(col("position")==1, True)).alias("wins"))\
            .orderBy(col("total_points").desc(),col("wins").desc())

In [0]:
from pyspark.sql.window import Window

window_spec= Window.partitionBy("race_year").orderBy(col("total_points").desc(),col("wins").desc())

final_df = constructor_standings_df.withColumn("rank", rank().over(window_spec))


In [0]:
#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation_db.constructor_standings")

#load_incremental_data(final_df, "race_year", "f1_presentation_db", "constructor_standings")

merge_condition = "target_tab.team = source_tab.team AND target_tab.race_year = source_tab.race_year"

merge_incremental_data(final_df, presentation_container_folder_path, merge_condition, "race_year", "f1_presentation_db", "constructor_standings")